In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 13 17:14:17 2021

@author: tasfi
"""

import matplotlib
matplotlib.use("Agg")
from matplotlib import colors, ticker, patches
from matplotlib import patheffects, gridspec
import matplotlib.dates as mdates
import numpy as np
import matplotlib.pyplot as plt
import iris # minimal fio done here 
import iris.quickplot as qplt
import cartopy.crs as ccrs
from datetime import datetime, timedelta
import numpy as np
from netCDF4 import Dataset
#from oceans import wrap_lon180
import oceans
import mplleaflet
#import Leaflet as L
import pandas as pd 
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import branca.colormap
from collections import defaultdict
import webbrowser
#import pylab
#import time
#from numba import jit, cuda 
wind_u = iris.load("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/10m_uwind.20200102T1500Z.nc")
wind_v = iris.load("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/10m_vwind.20200102T1500Z.nc")
#fire = iris.load("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/firefront.20200102T1500Z.nc")

#ro1_cubes = iris.load("C:/PHD/APR-BOM/Interactive-Map-Visualization-using-Folium-main/sampledata_ro1.nc")
#ro1_cubes = iris.load("C:/PHD/APR-BOM/visualize/my codes/umnsaa_2020010215_mdl_ro1.nc")


Bad key "text.kerning_factor" on line 4 in
C:\Users\tshermin\AppData\Local\Continuum\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [137]:
import math
import folium
import branca
from folium import plugins
from scipy.interpolate import griddata
import geojsoncontour
import scipy as sp
import scipy.ndimage

#y_wind1 = ro1_cubes.extract("y_wind")[0]
#x_wind = x_wind0[0,0]   #######x_wind0[time-dim,level-dim]

#create map
map0 = folium.Map(location=[-35.7, 136.4],#tiles="Cartodb Positron",
                  zoom_start=10)

#legend
steps=30

colormap = branca.colormap.LinearColormap(['blue', 'cyan', 'green', 'yellow','orange', 'red'],
                               index=[0, .2,.4, .6, .8,.95])

colormap0 = branca.colormap.LinearColormap(colors=['cyan','yellow','BlueViolet',
                                                   'lime','magenta','orange'
                                                   ,'red','green','indigo',
                                                  'brown'],
                                          index=[0,3,6,9,12,15,18,21,24,27],vmin=0,vmax=27
                                            ,caption='Wind Speed')#.to_step(steps)
colormap1 = branca.colormap.LinearColormap(colors=['cyan','yellow','BlueViolet',
                                                   'lime','magenta','springGreen'
                                                   ,'teal','green','indigo',
                                                  'brown'],
                                          index=[0,3,6,9,12,15,18,21,24,27],vmin=0,vmax=27
                                            ,caption='Wind Speed')#.to_step(steps)

colormap9 = branca.colormap.LinearColormap(colors=['Plum','violet','DarkOrchid',
                                                   'fuchsia','MediumOrchid','MediumPurple'
                                                   ,'BlueViolet','DarkMagenta','indigo',
                                                  'MediumSlateBlue'],
                                          index=[0,3,6,9,12,15,18,21,24,27],vmin=0,vmax=27
                                            ,caption='Wind Speed')#.to_step(steps)
colormap1._repr_html_()

gradient_map=defaultdict(dict)

for i in range(steps):
        gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
    
 #add color bar at the top of the map
#colormap1.add_to(map0)
#step.add_to(map0)


#wind direction quivers
def quiver1(x, y, U, V,feature_group0):
    fig, ax = plt.subplots()
       
    kw = dict(color='black', alpha=0.8, scale=1)
    q = ax.quiver(x, y, U, V, **kw)
    #plt.savefig("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/example_quiver.png")
    gj = mplleaflet.fig_to_geojson(fig=fig)

    count = 0
    count1 = 0
    b = 1
    #for 
    
    for feature in gj['features']: 
        #k=0
        if count1<=60 and b%6 == 0:
            if count%2==0:
                if feature['geometry']['type'] == 'Point':                
                    lon, lat = feature['geometry']['coordinates']
                #print(lat)   
                    #print(lon.shape)
                    div = feature['properties']['html']

                    icon_anchor = (feature['properties']['anchor_x'],
                                feature['properties']['anchor_y'])

                    icon = folium.features.DivIcon(html=div,
                                            icon_anchor=icon_anchor)
                    marker = folium.Marker([lat, lon], icon=icon)
                    feature_group0.add_child(marker)
        
            #else:
                #msg = "Unexpected geometry {}".format
                #raise ValueError(msg(feature['geometry']))
        if count1 == 60:
            count1 = 0
            b += 1
        #print(k)
        count += 1
        count1 += 1
    #fig.clf()
    #plt.ion()
    #plt.cla()
    #plt.clf()
    #plt.close(fig)
    #fig.clf()
    return feature_group0

# Setup minimum and maximum values for the contour lines

#@jit(target ="cuda") 
#def compute():

index = 1
k = 340
#fireii = firei.extract("firefront")[0] 
x_wind0 = wind_u.extract("UWIND_2")[0] # extract returns list of cubes
y_wind0 = wind_v.extract("VWIND_2")[0]

for i in range(index):
    #x_wind = x_wind0[0] #subset to first time
    #y_wind = y_wind0[0]
    if i%10==0:       
        #x_wind = wind_u.UWIND_2[k]
        #y_wind = wind_v.VWIND_2[k]
        x_wind = x_wind0[k] #subset to first time
        y_wind = y_wind0[k]
    
        lats,lons = x_wind.coord('latitude').points, x_wind.coord('longitude').points
    #print(lats)
        speed = (x_wind**2 + y_wind**2)**0.5
        print(np.max(speed.data))
 
    # Grid the elevation (Edited on March 30th, 2020)
        
    #print(speed)
    #speed = iris.analysis.maths.apply_ufunc(np.hypot,x_wind,y_wind)
    
        
        vmin = speed.data.min() 
        vmax = speed.data.max()
        print(vmax)
# Setup colormap
        color1 = ["MediumBlue","Blue","RoyalBlue","DodgerBlue","deepskyblue","aqua","LightSkyBlue","gold","tan","Aquamarine",
                    "yellow", "GreenYellow","Lime","LightGreen","MediumSeaGreen",
                    "SeaGreen", "ForestGreen","DarkGreen","teal","Orchid","Magenta","MediumPurple","BlueViolet",
                  "Indigo","slateblue","darkslateblue","mediumslateblue"#"MediumTurquoise","PapayaWhip"
                    ]
    #['blue','royalblue', 'navy','indigo','cyan','orchid','mediumpurple',  'darkorchid',  'plum',  'm', 'mediumvioletred',
                  #'BlueViolet', 
                  #'palevioletred', 'black','magenta','pink','MediumOrchid','yellow','lime', 'red','green', 'springGreen',
                  #'darkgreen','teal','DarkTurquoise','brown','white']
        step = branca.colormap.StepColormap(
         colors=['cyan','yellow','BlueViolet','lime','magenta','teal'
                                                   ,'DarkTurquoise','green','indigo',
                                                  'brown'],                                          
         vmin=0, vmax=27,
         index=[0,3,6,9,12,15,18,21,24,27],  #for change in the colors, not used fr linear
         caption='10m-Wind Speed'    #Caption for Color scale or Legend
            )
        levels = len(color1)
        cm     = branca.colormap.LinearColormap(color1, vmin=0, vmax=27).to_step(levels)
        
        contourf = plt.contourf(lons, lats, speed.data.T, levels, alpha=1, colors=color1,#cmap='viridis',# 
                                linestyles='None', vmin=vmin, vmax=vmax)
        geojson = geojsoncontour.contourf_to_geojson(
        contourf=contourf,
        min_angle_deg=3.0,
        ndigits=5,
        stroke_width=1,
        fill_opacity=1)
    
        feature_group0 = folium.FeatureGroup(name='quiver')
        u = x_wind
        v = y_wind
        lon = lons
        lat = lats
        lat, lon = np.meshgrid(lat, lon)
        mask_x = np.logical_and(lon > 136, lon < 138)
    #print(mask_x)
        mask_y = np.logical_and(lat > -38, lat < -35)
    #print(mask_y)
        mask = np.logical_and(mask_x, mask_y)
        #print(mask.shape)
    
        x = lon[mask]
        y = lat[mask]
        u.data = u.data/128 #reduced the vectors by power of 2 for reducing browser load
        v.data = v.data/128
        U = u.data[mask]#.filled(fill_value=0)
        V = v.data[mask]#.filled(fill_value=0)
        feature_group0 = quiver1(x, y, U, V,feature_group0)      
        map0.add_child(feature_group0)
        plt.clf()    
        
        folium.GeoJson(
        geojson,
        style_function=lambda x: {
        'color':     x['properties']['stroke'],
        'weight':    x['properties']['stroke-width'],
        'fillColor': x['properties']['fill'],
        'opacity':   0.5,
        }).add_to(map0)
        
    
        
        print(i)
        map0.add_child(cm)

        #map0.add_child(folium.map.LayerControl())

        #map0.save(r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\figures\filled contour wind speed heat flux\WSDHF_' + str(k) + '.html')
        #map0.save (r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\fire\windSpeednDirection.html')
        #webbrowser.open(r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\figures\wind speed n direction\windSpeednDirection' + str(i) + '.html')
#map0

16.049011999829666
16.049011999829666
0


In [138]:
from matplotlib import colors, ticker, patches
sen_heat = iris.load("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/sensible_heat.20200102T1500Z.nc")
#print(sen_heat)
fire = iris.load("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/firefront.20200102T1500Z.nc")
ff = fire.extract("firefront")[0]
#print(wind_u)
#print((sen_heat[0].data))
#print(np.min(sen_heat[0].data))
#print('\n')
#print(wind_v)
sen_heati = sen_heat.extract("SHEAT_2")[0] 
S_heat = sen_heati[0]
#print(np.min(sen_heati.data))
#print(np.max(S_heat.data))


fflux = sen_heat[0]#sen_heati
#fflux = fflux[::20]
#nt,ny,nx = fflux.shape

lats,lons = ff.coord('latitude').points, ff.coord('longitude').points
# helper function to set up tiff image backdrop
backdrop = "badja.tiff"

#fig,ax = plotting.map_tiff_qgis(fname=backdrop,)

#ax = plt.axes(projection=ccrs.PlateCarree())

#fig1, ax1 = plt.subplots(constrained_layout=True)
# set up colorbar scaling (norm and levels)
index_min,index_max=2,5
fflux_min,fflux_max=10**index_min,10**index_max
fflux_norm = colors.LogNorm(vmin=fflux_min,vmax=fflux_max)
fflux_levels = np.logspace(index_min,index_max,20)
index = 1
for i in range(index):
    if i%10==0:       
        #x_wind = wind_u.UWIND_2[k]
        #y_wind = wind_v.VWIND_2[k]
        ffluxti = fflux[k].data
        if np.max(ffluxti) > fflux_min:
            cs=plt.contourf(lons,lats,ffluxti.T+1, fflux_levels,
                    cmap='hot',norm=fflux_norm, 
                    vmin=fflux_min, vmax=fflux_max,
                    alpha=1,
                    extend='max',
                    )
            geojson1 = geojsoncontour.contourf_to_geojson(
            contourf=cs,
            min_angle_deg=30.0,
            ndigits=5,
            stroke_width=1.0,
            fill_opacity=1)
 
    # Set up the map placeholdder
#map0 = folium.Map(location=[-35.5, 136.4],#tiles="Cartodb Positron",
                  #zoom_start=9)
#geomap1 = folium.Map([DEM.latitude.mean(), DEM.longitude.mean()], zoom_start=12, tiles="OpenStreetMap")
# Plot the contour on Folium map
            plt.clf() 
            folium.GeoJson(
            geojson1,
            style_function=lambda x: {
            'color':     x['properties']['stroke'],
            'weight':    x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity':   0.5,
                }).add_to(map0)
        
map0.save(r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\figures\filled contour wind speed heat flux\WSDHF_' + str(k) + '.html')
        
map0

In [4]:
import folium
import branca
from folium import plugins
from scipy.interpolate import griddata
import geojsoncontour
import scipy as sp
import scipy.ndimage
# Convert matplotlib contourf to geojson
geojson1 = geojsoncontour.contourf_to_geojson(
    contourf=cs,
    min_angle_deg=0.0,
    ndigits=5,
    stroke_width=1.0,
    fill_opacity=0.1)
 
# Set up the map placeholdder
#map0 = folium.Map(location=[-35.5, 136.4],#tiles="Cartodb Positron",
                  #zoom_start=9)
#geomap1 = folium.Map([DEM.latitude.mean(), DEM.longitude.mean()], zoom_start=12, tiles="OpenStreetMap")
# Plot the contour on Folium map
plt.clf() 
folium.GeoJson(
    geojson1,
    style_function=lambda x: {
        'color':     x['properties']['stroke'],
        'weight':    x['properties']['stroke-width'],
        'fillColor': x['properties']['fill'],
        'opacity':   0.5,
    }).add_to(map0)
 
# Add the colormap to the folium map for legend
#cm.caption = 'Elevation'
#geomap1.add_child(cm)
 
# Add the legend to the map
#plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap1)

map0.save(r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\figures\filled contour wind speed heat flux\WSDHF_' + str(k) + '.html')
        
map0

NameError: name 'cs' is not defined

In [ ]:
map0.save(r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\figures\filled contour wind speed heat flux\WSDHF_' + str(k) + '.html')
        